<H1>ФБ-31 Федорович Дарина.</H1>
<H1>Лабараторна робота №2: Наука про дані: підготовчий етап</H1>

Налаштування середовища для даної лабораторної роботи 

In [5]:
import datetime
import os
import pandas as pd
import glob


print("Setup Complete")

Setup Complete


Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу.

In [1]:
import os
import urllib.request
import datetime

def download_data(province_id, year_start=1981, year_end=2025, overwrite=False):
    # Створення папки 'dataVHI', якщо вона не існує
    data_folder = 'dataVHI'
    os.makedirs(data_folder, exist_ok=True)

    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    filename = f'vhi_id__{province_id}__{current_datetime}.csv'
    file_path = os.path.join(data_folder, filename)

    existing_files = [file for file in os.listdir(data_folder) if file.startswith(f'vhi_id__{province_id}__')]
    if existing_files and not overwrite:
        print(f"Файл '{existing_files[0]}' вже існує. Завантаження пропущено.")
        return

    
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year_start}&year2={year_end}&type=Mean"
    
    try:
        # Виконуємо запит
        print(f"Завантаження даних з {url} ...")
        response = urllib.request.urlopen(url)
        
        # Перевіряємо статус відповіді
        if response.status != 200:
            print(f"Помилка: не вдалося завантажити файл (HTTP {response.status})")
            return

        # Збереження у файл
        with open(file_path, 'wb') as file:
            file.write(response.read())

        # Видаляємо старі файли, залишаючи тільки новий
        for old_file in existing_files:
            os.remove(os.path.join(data_folder, old_file))
        
        print(f"Дані успішно завантажено та збережено: '{file_path}'")
    
    except Exception as e:
        print(f"Помилка під час завантаження: {e}")

In [7]:
for i in range(1,28):
   download_data(i)

Файл 'vhi_id__1__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__2__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__3__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__4__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__5__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__6__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__7__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__8__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__9__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__10__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__11__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__12__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__13__2025-02-18_17-36.csv' вже існує. Завантаження пропущено.
Файл 'vhi_id__14__2025-02-18_17-36

Встановлення відповідності між id регіону та його назвою за алфавітом

In [8]:
province_map = {
    1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
    6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
    11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
    16: 'Рівненська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
    21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
}

Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [10]:
def create_vhi_dataframe(data_folder):
    csv_files = glob.glob(data_folder + "/*.csv")

    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'extra']
    dataframes = []

    for csv_file in csv_files:
        region_code = int(csv_file.split('__')[1])  
        df = pd.read_csv(csv_file, header=1, names=columns)

        df.at[0, 'Year'] = df.at[0, 'Year'][9:]  
        df = df.drop(df.index[-1])  
        df = df[df['VHI'] != -1]  
        df = df.drop('extra', axis=1)

        df.insert(0, 'region_code', region_code, True)
        df['Week'] = df['Week'].astype(int)
        
        dataframes.append(df)   
        
    combined_df = pd.concat(dataframes).drop_duplicates().reset_index(drop=True)

    combined_df = combined_df.loc[(combined_df.region_code != 12) & (combined_df.region_code != 20)]
    combined_df = combined_df.replace({'region_code': {1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21, 
                                                       11:9, 13:10, 14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 21:17, 
                                                       22:18, 23:6, 24:1, 25:2, 26:6, 27:5}})
    
    return combined_df


In [11]:
result = create_vhi_dataframe('dataVHI')

print(result)

       region_code  Year  Week    SMN     SMT    VCI    TCI    VHI
0               21  1982     1  0.059  258.24  51.11  48.78  49.95
1               21  1982     2  0.063  261.53  55.89  38.20  47.04
2               21  1982     3  0.063  263.45  57.30  32.69  44.99
3               21  1982     4  0.061  265.10  53.96  28.62  41.29
4               21  1982     5  0.058  266.42  46.87  28.57  37.72
...            ...   ...   ...    ...     ...    ...    ...    ...
59179           20  2025     2  0.125  276.09  70.85   1.96  36.40
59180           20  2025     3  0.121  275.76  71.09   2.30  36.69
59181           20  2025     4  0.119  275.86  71.56   3.75  37.64
59182           20  2025     5  0.122  276.27  71.94   4.58  38.25
59183           20  2025     6  0.128  276.56  71.39   8.48  39.94

[54800 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду
<pre>o Ряд VHI для області за вказаний рік;</pre>

In [12]:
def get_vhi_by_region_and_year(df, region_code, year):
 
    filtered_data = df[(df['region_code'] == region_code) & (df['Year'].astype(int) == year)]
    return filtered_data[['Year', 'Week', 'VHI']]

# Використання:
region_id = 10 
year = 2025

vhi_series = get_vhi_by_region_and_year(result, region_id, year)
print(vhi_series)


      Year  Week    VHI
8762  2025     1  38.58
8763  2025     2  39.88
8764  2025     3  41.38
8765  2025     4  43.46
8766  2025     5  44.50
8767  2025     6  46.11


<pre>o Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;</pre>

In [13]:
def get_vhi_statistics(df, region_code, year):
    filtered_data = df[(df['region_code'] == region_code) & (df['Year'].astype(int) == year)]
    
    if filtered_data.empty:
        return f"Дані для регіону {region_code} за {year} рік відсутні."
    
    vhi_values = filtered_data['VHI']
    
    stats = {
        'Мінімальний VHI': round(float(vhi_values.min()), 2),
        'Максимальний VHI': round(float(vhi_values.max()), 2),
        'Середній VHI': round(float(vhi_values.mean()), 2),
        'Медіана VHI': round(float(vhi_values.median()), 2)
    }
    
    return stats

# Приклад використання:
region_id = 10 
year = 2024

vhi_stats = get_vhi_statistics(result, region_id, year)
print(vhi_stats)

{'Мінімальний VHI': 21.08, 'Максимальний VHI': 49.51, 'Середній VHI': 38.52, 'Медіана VHI': 38.8}


<pre>o Ряд VHI за вказаний діапазон років для вказаних областей</pre>

In [14]:
def get_vhi_by_regions_and_year_range(df, region_codes, start_year, end_year):
    filtered_data = df[
        (df['region_code'].isin(region_codes)) & 
        (df['Year'].astype(int).between(start_year, end_year))
    ]
    
    return filtered_data[['region_code', 'Year', 'Week', 'VHI']].sort_values(by=['region_code', 'Year', 'Week'])

# Приклад використання:
selected_regions = [3, 12]  
start_year = 2022
end_year = 2024

vhi_time_series = get_vhi_by_regions_and_year_range(result, selected_regions, start_year, end_year)
print(vhi_time_series)

       region_code  Year  Week    VHI
50254            3  2022     1  51.23
50255            3  2022     2  49.10
50256            3  2022     3  47.49
50257            3  2022     4  46.99
50258            3  2022     5  45.43
...            ...   ...   ...    ...
13141           12  2024    48  44.15
13142           12  2024    49  42.39
13143           12  2024    50  39.90
13144           12  2024    51  40.62
13145           12  2024    52  42.24

[312 rows x 4 columns]


<pre>o Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка 
областей по Україні (20%областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами 
та значення VHI</pre>

In [15]:
def detect_drought_years(df, vhi_threshold=15, affected_percentage=20):
    total_regions = df['region_code'].nunique()
    min_affected_regions = int((affected_percentage / 100) * total_regions)

    drought_years = {}

    for year in df['Year'].unique():
        yearly_data = df[df['Year'] == year]

        region_droughts = yearly_data.groupby('region_code')['VHI'].min()
        affected_regions = region_droughts[region_droughts <= vhi_threshold]

        affected_count = len(affected_regions) 
        affected_percent = (affected_count / total_regions) * 100 

        if affected_count >= min_affected_regions:
            drought_years[int(year)] = (round(affected_percent, 2), affected_regions.to_dict())

    return drought_years


# Використання:
drought_data = detect_drought_years(result, vhi_threshold=15, affected_percentage=20)

for year, (percent, regions) in drought_data.items():
    print(f"{year} (Посуха у {percent:.2f}% областей):")
    for region_id, vhi_value in regions.items():
        region_name = province_map.get(region_id, f"Область {region_id}") 
        print(f"{region_name}: VHI = {vhi_value:.2f}")


2000 (Посуха у 16.67% областей):
Вінницька: VHI = 11.25
Київська: VHI = 10.60
Харківська: VHI = 9.36
Черкаська: VHI = 10.68
2007 (Посуха у 20.83% областей):
Закарпатська: VHI = 10.88
Миколаївська: VHI = 5.94
Одеська: VHI = 5.52
Херсонська: VHI = 12.23
Республіка Крим: VHI = 13.28
